# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [32]:
import pandas as pd

In [33]:
# your code here

tweets = pd.read_csv('training.1600000.processed.noemoticon.csv')
tweets.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
tweets.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [34]:
sample_tweets = tweets.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [35]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

In [36]:
# your code here

def clean_up(s):

    string_no_url = re.sub('http://.+', ' ', s)
    string_no_special = re.sub('\W', ' ', string_no_url)
    string_no_digits = re.sub('\d', ' ', string_no_special)
    string = string_no_digits.lower()
    
    return string

def tokenize(s):

    token = word_tokenize(s)
    
    return token

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def stem_and_lemmatize(l):

    #stem = PorterStemmer()
    #stemmed = [stem.stem(word) for word in l]
    
    lem = WordNetLemmatizer()
    lemmed = [lem.lemmatize(word, get_wordnet_pos(word)) for word in l]
    
    return lemmed #, stemmed

stop = stopwords.words('english')

def remove_stopwords(row):

    without_sw = [word for word in row if not word in stop]
    
    return without_sw

In [37]:
sample_tweets['text_processed'] = sample_tweets['text'].apply(clean_up)
sample_tweets['text_processed'] = sample_tweets['text_processed'].apply(tokenize)
sample_tweets['text_processed'] = sample_tweets['text_processed'].apply(stem_and_lemmatize)

In [38]:
sample_tweets['text_processed'] = sample_tweets['text_processed'].apply(remove_stopwords)

sample_tweets.head()

,target,id,date,flag,user,text,text_processed
880490,4,1685644097,Sun May 03 01:21:02 PDT 2009,NO_QUERY,lisayamaguchi,and we got married,"[get, married]"
551696,0,2203227101,Tue Jun 16 23:17:06 PDT 2009,NO_QUERY,Meganlynn11,Rcct meeting tomorrow. It will be fun! Last da...,"[rcct, meeting, tomorrow, fun, last, day, see,..."
1090081,4,1969765423,Sat May 30 01:08:41 PDT 2009,NO_QUERY,cymberrain,@misskely you have a better rest of the day,"[misskely, well, rest, day]"
233561,0,1979397967,Sun May 31 02:52:31 PDT 2009,NO_QUERY,romyskye,@Ranting_Llama True but it felt so amazing to ...,"[ranting_llama, true, felt, amaze, give, sat, ..."
1586993,4,2190881632,Tue Jun 16 03:59:02 PDT 2009,NO_QUERY,coolhyperion,"@madsdeguzman will treat you to haagen daaz, b...","[madsdeguzman, treat, haagen, daaz, come, get,..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [39]:
from nltk.probability import FreqDist

In [41]:
# your code here

all_words = []

for row in sample_tweets.text_processed:
    for word in row:
        all_words.append(word)
        
fdist = nltk.FreqDist(all_words)
top = dict(fdist.most_common())
top = list(top.keys())
top = top[:5000]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [42]:
# your code here

document = sample_tweets['text_processed']

def find_features(document):
    words = set(document)
    features = {}
    for word in top:
        features[word] = (word in words)
    
    return features

In [52]:
feature_sets = [(find_features(tweet), target) for (tweet, target) in list(zip(sample_tweets['text_processed'], sample_tweets['target']))]
#print(len(feature_sets))

In [44]:
len(X)

20000

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [53]:
training_set = feature_sets[:15000]
testing_set = feature_sets[5000:]

In [55]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print('NiveByies algorightm accuracy is:', nltk.classify.accuracy(classifier, testing_set))
classifier.show_most_informative_features(15)

NiveByies algorightm accuracy is: 0.7832
Most Informative Features
                    sigh = True                0 : 4      =     20.5 : 1.0
                    scar = True                0 : 4      =     19.2 : 1.0
                  throat = True                0 : 4      =     15.8 : 1.0
                   yummy = True                4 : 0      =     14.8 : 1.0
                     idk = True                0 : 4      =     14.5 : 1.0
               fantastic = True                4 : 0      =     14.2 : 1.0
                    ache = True                0 : 4      =     13.8 : 1.0
                    poor = True                0 : 4      =     12.7 : 1.0
                  broken = True                0 : 4      =     12.3 : 1.0
                     sad = True                0 : 4      =     11.8 : 1.0
                  afford = True                0 : 4      =     11.8 : 1.0
                     www = True                4 : 0      =     11.7 : 1.0
                     fml = True  

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [56]:
# your code here

nltk.classify.accuracy(classifier, testing_set)

0.7832

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here